In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.insert(1, os.path.join(sys.path[0], '../..')) # Add parent directory to path

## Set GPUs

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch
print(torch.cuda.is_available())

from utils import read_yaml
from parse_arguments import ConfigParser
from train import set_up_trainer

True


CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.


CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
Pykeops installation found.


## Load Config

In [3]:
fname = r'/home/leo/QuantisedSSM/S4/configs/S4D_small.yaml'
# resume_checkpoint = r"/home/leo/QuantisedSSM/S4/log/S4D_small/fp32_16heads/checkpoint/ckpt.pth"
resume_checkpoint = None

config_yaml = read_yaml(fname)
# modification = {"model;type": 32}
modification = None
config = ConfigParser(config_yaml, resume_checkpoint, modification=modification, save_log=False)

## Set up trainer

In [6]:
trainer = set_up_trainer(config)

[*] Generating MNIST Classification Dataset...
[*] Train set size: 50000
[*] Validation set size: 10000
[*] Test set size: 10000
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 784, 16]             32
├─ModuleList: 1                          []                        --
|    └─S4Block: 2-1                      [-1, 16, 784]             --
|    |    └─FFTConv: 3-1                 [-1, 784, 16]             3,104
|    |    └─Identity: 3-2                [-1, 784, 16]             --
|    |    └─Identity: 3-3                [-1, 784, 16]             --
|    |    └─Sequential: 3-4              [-1, 784, 16]             544
├─ModuleList: 1                          []                        --
|    └─Dropout1d: 2-2                    [-1, 16, 784]             --
├─Linear: 1-2                            [-1, 10]                  170
Total params: 3,850
Trainable params: 3,850
Non-trainable params: 0
Total mult-adds (M): 0.

## Train and Eval

In [26]:
trainer.train()
trainer.eval()

0it [00:00, ?it/s]

TRAIN - Batch Idx: (97/98) | Loss: 1.534 | Acc: 59.606% (29803/50000): : 98it [00:26,  3.76it/s]
VAL - Batch Idx: (19/20) | Loss: 0.254 | Acc: 92.600% (9260/10000): : 20it [00:02,  7.73it/s]


Best model found. Saving checkpoint at /home/leo/QuantisedSSM/S4/log/S4D_small/0207_104603


TEST - Batch Idx: (19/20) | Loss: 0.241 | Acc: 92.790% (9279/10000): : 20it [00:02,  7.79it/s]


Epoch 1/30, Train Acc: 59.606, Val Acc: 92.6, Test Acc: 92.79, LR: [0.009938441702975689]


...

TRAIN - Batch Idx: (97/98) | Loss: 0.030 | Acc: 99.070% (49535/50000): : 98it [00:26,  3.75it/s]
VAL - Batch Idx: (19/20) | Loss: 0.046 | Acc: 98.640% (9864/10000): : 20it [00:02,  7.69it/s]
TEST - Batch Idx: (19/20) | Loss: 0.037 | Acc: 98.960% (9896/10000): : 20it [00:02,  7.74it/s]


Epoch 30/30, Train Acc: 99.07, Val Acc: 98.64, Test Acc: 98.96, LR: [0.004999999999999998]


TEST - Batch Idx: (19/20) | Loss: 0.037 | Acc: 98.960% (9896/10000): : 20it [00:02,  7.74it/s]

Eval Acc: 98.96, Loss: 0.03668064445373602


{'loss': 0.03668064445373602, 'acc': 98.96}

In [3]:
# LOOP: run on multiple seeds

fname = r'/home/leo/QuantisedSSM/S4/log/S4D_small/fp32_512heads/config.yaml'
# resume_checkpoint = r"/home/leo/QuantisedSSM/S4/log/S4D_small/fp32_16heads/checkpoint/ckpt.pth"
resume_checkpoint = None

config_yaml = read_yaml(fname)

for seed in range(5):
    modification = {"seed": seed}
    config = ConfigParser(config_yaml, resume_checkpoint, modification=modification, save_log=True)

    trainer = set_up_trainer(config)
    trainer.train()
    trainer.eval()

    del trainer, config

[*] Generating MNIST Classification Dataset...
[*] Train set size: 50000
[*] Validation set size: 10000
[*] Test set size: 10000
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 784, 512]            1,024
├─ModuleList: 1                          []                        --
|    └─S4Block: 2-1                      [-1, 512, 784]            --
|    |    └─FFTConv: 3-1                 [-1, 784, 512]            99,328
|    |    └─Identity: 3-2                [-1, 784, 512]            --
|    |    └─Identity: 3-3                [-1, 784, 512]            --
|    |    └─Sequential: 3-4              [-1, 784, 512]            525,312
├─ModuleList: 1                          []                        --
|    └─Dropout1d: 2-2                    [-1, 512, 784]            --
├─Linear: 1-2                            [-1, 10]                  5,130
Total params: 630,794
Trainable params: 630,794
Non-trainable params: 0
Total mul

TRAIN - Batch Idx: (97/98) | Loss: 1.534 | Acc: 59.606% (29803/50000): : 98it [00:26,  3.74it/s]
VAL - Batch Idx: (19/20) | Loss: 0.254 | Acc: 92.600% (9260/10000): : 20it [00:02,  7.73it/s]


Best model found. Saving checkpoint at /home/leo/QuantisedSSM/S4/log/S4D_small/0403_180315


TEST - Batch Idx: (19/20) | Loss: 0.241 | Acc: 92.790% (9279/10000): : 20it [00:02,  7.72it/s]


Epoch 1/30, Train Acc: 59.606, Val Acc: 92.6, Test Acc: 92.79, LR: [0.009938441702975689]


...

Epoch 29/30, Train Acc: 99.192, Val Acc: 98.85, Test Acc: 99.04, LR: [0.004217827674798845]


TRAIN - Batch Idx: (97/98) | Loss: 0.030 | Acc: 98.994% (49497/50000): : 98it [00:26,  3.75it/s]
VAL - Batch Idx: (19/20) | Loss: 0.046 | Acc: 98.680% (9868/10000): : 20it [00:02,  7.69it/s]
TEST - Batch Idx: (19/20) | Loss: 0.037 | Acc: 98.850% (9885/10000): : 20it [00:02,  7.72it/s]


Epoch 30/30, Train Acc: 98.994, Val Acc: 98.68, Test Acc: 98.85, LR: [0.004999999999999998]


TEST - Batch Idx: (19/20) | Loss: 0.037 | Acc: 98.850% (9885/10000): : 20it [00:02,  7.74it/s]

Eval Acc: 98.85, Loss: 0.03675868332211394


## Quantise

In [23]:
from quantize import set_up_quantizers
from pathlib import Path

model_path = Path(r"/home/leo/QuantisedSSM/S4/log/S4D_small/fp32_16heads_seed0")
fname = model_path / "config.yaml"
resume_checkpoint =  model_path / "checkpoint" / "ckpt.pth"
config_yaml = read_yaml(fname)

# q_config_fname = r"/home/leo/QuantisedSSM/S4/configs/q_config.yaml"
q_config_fname = r"/home/leo/QuantisedSSM/S4/configs/q_config_conv.yaml"
q_config_yaml = read_yaml(q_config_fname)
q_config = ConfigParser(q_config_yaml, None, modification=None, save_log=False)

modification = {"run_name": q_config["run_name"]}

resume_checkpoint =  r"/home/leo/QuantisedSSM/S4/log/S4D_small/fp32_16heads_seed0/checkpoint/ckpt.pth"
config = ConfigParser(config_yaml, resume_checkpoint, modification=modification, save_log=False)

q_trainer = set_up_trainer(config)
set_up_quantizers(q_trainer, q_config)

q_trainer.model

/home/leo/QuantisedSSM/S4/notebooks/../../S4/quantization/q_modules/qS4D_convolutional.py:121: SyntaxWarning: invalid escape sequence '\s'
  """


CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
Pykeops installation found.
[*] Generating MNIST Classification Dataset...
[*] Train set size: 50000
[*] Validation set size: 10000
[*] Test set size: 10000
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 784, 16]             32
├─ModuleList: 1                          []                        --
|    └─S4Block: 2-1                      [-1, 16, 784]             --
|    |    └─FFTConv: 3-1                 [-1, 784, 16]             3,104
|    |    └─Identity: 3-2                [-1, 784, 16]             --
|    |    └─Identity: 3-3                [-1, 784, 16]             --
|    |    └─Sequential: 3-4              [-1, 784, 16]       

/home/leo/QuantisedSSM/S4/notebooks/../../train.py:90: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint)


S4DModel(
  (encoder): Sequential(
    (0): ActivationQuantizer(self.fake_real='fake', self.num_bits=8, self.dims_to_reduce=None, self.percentile=99.999, self.mode='asymmetric', self.zero_point_bits=16)
    (1): QLinear()
  )
  (s4_layers): ModuleList(
    (0): Sequential(
      (0): ActivationQuantizer(self.fake_real='fake', self.num_bits=8, self.dims_to_reduce=[-2], self.percentile=99.999, self.mode='asymmetric', self.zero_point_bits=16)
      (1): S4BlockConvolutional(
        (layer): FFTConv(
          (activation): QGELU()
          (kernel): SSMKernelDiag()
          (drop): Identity()
          (drop_kernel): Identity()
        )
        (mult_activation): Identity()
        (drop): Identity()
        (output_linear): Sequential(
          (0): Sequential(
            (0): ActivationQuantizer(self.fake_real='fake', self.num_bits=8, self.dims_to_reduce=None, self.percentile=99.999, self.mode='asymmetric', self.zero_point_bits=16)
            (1): QLinear()
          )
          

In [24]:
conv_layer = q_trainer.model.s4_layers[0][1]

# Define a forward hook for the S4BlockRecurrent module. 
# Store the input and output activations in a dictionary.
def record_statistics(record_dict):
    def get_statistics(module, input, output):
        _, (inputs, post_GELU_activation, pre_GELU_activation, all_states) = output
        record_dict["input_activation"] = inputs.detach() if inputs is not None else None
        record_dict["output_activation"] = post_GELU_activation.detach() if post_GELU_activation is not None else None
        record_dict["output_pre_activation"] = pre_GELU_activation.detach() if pre_GELU_activation is not None else None
        record_dict["all_states"] = all_states.detach() if all_states is not None else None
    
    return get_statistics

# Get activation statistics for a single batch. Can activate/deactivate quantisation before getting statistics.
def get_activation_statistics_dict(q_trainer, recur_layer):
    q_record_dict = {}
    hook_handle = recur_layer.register_forward_hook(record_statistics(q_record_dict))
    batch, labels = next(iter(q_trainer.trainloader))
    q_trainer.model(batch.to(q_trainer.device))
    hook_handle.remove()
    return q_record_dict


# conv_layer.enable_quantization()
conv_layer.disable_quantization()
conv_layer.disable_quantization_flag 
# from S4.quantization.q_modules import qModules
# qModules.S4BlockConvolutional

q_record_dict = get_activation_statistics_dict(q_trainer, conv_layer)

Disabled quantization for S4BlockConvolutional


AttributeError: 'NoneType' object has no attribute 'detach'

In [22]:
# Dynamically quantised: default
q_trainer.eval()

0it [00:00, ?it/s]

TEST - Batch Idx: (19/20) | Loss: 0.093 | Acc: 97.220% (9722/10000): : 20it [00:01, 18.49it/s]

Eval Acc: 97.22, Loss: 0.09282875079661608


{'loss': 0.09282875079661608, 'acc': 97.22}

In [4]:
from torch.profiler import profile, record_function, ProfilerActivity

# torch.cuda.empty_cache()
image, label = next(iter(q_trainer.testloader))
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof:
    q_trainer.model(image.to(q_trainer.device))

[W124 10:16:58.655094641 kineto_shim.cpp:415] Adding profiling metadata requires using torch.profiler with Kineto support (USE_KINETO=1)
[W124 10:16:58.655114609 kineto_shim.cpp:415] Adding profiling metadata requires using torch.profiler with Kineto support (USE_KINETO=1)


In [5]:
print(prof.key_averages().table(sort_by="cuda_memory_usage", row_limit=10))

# image, label = next(iter(q_trainer.testloader))
# q_trainer.model(image.to(q_trainer.device))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   aten::sort         5.09%     190.375ms         7.27%     272.055ms     172.405us     179.002ms         4.60%     259.209ms     164.264us           0 b           0 b       1.78 Gb       1.78 Gb          1578  
                    aten::abs         1.25%      46.701ms         2.11%      78.786ms 

In [54]:
# Eval
# results_log = q_trainer.eval()

# Quantize activations statically
q_trainer.calibrate(num_batches=1)
# q_trainer.set_static_quantization()
# q_trainer.set_dynamic_quantization()
# results_log = q_trainer.eval()


Starting calibration with 1 batches
Disabling quantization for 0
Disabled quantization for activation quantizer
Disabling quantization for 1
Disabling quantization for 0
Disabled quantization for activation quantizer
Disabling quantization for 1
Disabled quantization for S4BlockRecurrent
Disabling quantization for layer_activation
Disabling quantization for 0
Disabled quantization for activation quantizer
Disabling quantization for 1
Disabling quantization for state_quantizer
Disabled quantization for activation quantizer
Disabling quantization for y_recur_quantizer
Disabled quantization for activation quantizer
Disabling quantization for y_pre_output_quantizer
Disabled quantization for activation quantizer
Disabling quantization for 0
Disabled quantization for activation quantizer
Disabling quantization for 1
Set up collecting statistics for activation quantizer
Starting statistics collection for 0
Set up collecting statistics for activation quantizer
Starting statistics collection fo

In [53]:
q_trainer.model.s4_layers[0][0].amax 

tensor([[[ 4.7342],
         [ 1.2082],
         [-0.0606],
         [ 5.0070],
         [ 0.4697],
         [-0.6930],
         [ 6.1079],
         [ 1.0016],
         [ 5.0402],
         [-0.0769],
         [ 1.4305],
         [ 1.6072],
         [ 6.0985],
         [ 3.7426],
         [ 4.1993],
         [ 3.8153]]], device='cuda:0')

In [4]:
# QAT
train_log = q_trainer.train_epoch(1)
eval_log = q_trainer.eval()

0it [00:00, ?it/s]

TRAIN - Batch Idx: (7/391) | Loss: 20.838 | Acc: 12.207% (125/1024): : 8it [00:34,  4.26s/it]


KeyboardInterrupt: 

In [23]:
# Make sure still quantised
s4_block = q_trainer.model.s4_layers[0][1]
torch.unique(torch.view_as_real(s4_block.dA)[15,:,:])
# Check activations are quantised, redo static quantisation
q_trainer.calibrate(num_batches=1)
q_trainer.eval()

Starting calibration with 1 batches
Disabling quantization for 0
Disabled quantization for activation quantizer
Disabling quantization for 1
Disabled quantization for S4BlockRecurrent
Disabling quantization for state_quantizer
Disabled quantization for activation quantizer
Disabling quantization for y_recur_quantizer
Disabled quantization for activation quantizer
Disabling quantization for y_pre_output_quantizer
Disabled quantization for activation quantizer
Set up collecting statistics for activation quantizer
Starting statistics collection for 0
Set up collecting statistics for activation quantizer
Starting statistics collection for state_quantizer
Set up collecting statistics for activation quantizer
Starting statistics collection for y_recur_quantizer
Set up collecting statistics for activation quantizer
Starting statistics collection for y_pre_output_quantizer
Ending statistics collection for 0
Finished collecting statistics for activation quantizer
Updated amax and amin for activ

TEST - Batch Idx: (78/79) | Loss: 0.768 | Acc: 76.440% (7644/10000): : 79it [00:31,  2.55it/s]

Eval Acc: 76.44, Loss: 0.7676162231194822


{'loss': 0.7676162231194822, 'acc': 76.44}

## Test loading from sweep data

In [56]:
# Test loading from sweep data

from quantize import set_up_quantizers
from pathlib import Path

model_path = Path(r"/home/leo/QuantisedSSM/S4/log/sweep_asymmetric_symmetric/asymmetric_8bits_all_weights_acts_perchannel_seed_0/QAT_9_static")
fname = model_path.parents[1] / "config_seed0.yaml"
resume_checkpoint =  model_path / "checkpoint" / "ckpt.pth"

config_yaml = read_yaml(fname)

q_config_fname = model_path.parent / "q_config.yaml"
q_config_yaml = read_yaml(q_config_fname)
q_config = ConfigParser(q_config_yaml, resume_checkpoint, modification=None, save_log=False)

modification = {"run_name": q_config["run_name"]}

config = ConfigParser(config_yaml, None, modification=modification, save_log=False)

q_trainer = set_up_trainer(config)
set_up_quantizers(q_trainer.model, q_config)
# q_trainer.load_checkpoint(resume_checkpoint)

q_trainer.model

[*] Generating MNIST Classification Dataset...
[*] Train set size: 50000
[*] Validation set size: 10000
[*] Test set size: 10000
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 784, 16]             32
├─ModuleList: 1                          []                        --
|    └─S4Block: 2-1                      [-1, 16, 784]             --
|    |    └─FFTConv: 3-1                 [-1, 784, 16]             3,104
|    |    └─Identity: 3-2                [-1, 784, 16]             --
|    |    └─Identity: 3-3                [-1, 784, 16]             --
|    |    └─Sequential: 3-4              [-1, 784, 16]             544
├─ModuleList: 1                          []                        --
|    └─Dropout1d: 2-2                    [-1, 16, 784]             --
├─Linear: 1-2                            [-1, 10]                  170
Total params: 3,850
Trainable params: 3,850
Non-trainable params: 0
Total mult-adds (M): 0.

S4DModel(
  (encoder): Sequential(
    (0): ActivationQuantizer(self.fake_real='fake', self.num_bits=8, self.dims_to_reduce=None, self.percentile=99.999, self.mode='asymmetric', self.zero_point_bits=16)
    (1): QLinear()
  )
  (s4_layers): ModuleList(
    (0): Sequential(
      (0): ActivationQuantizer(self.fake_real='fake', self.num_bits=8, self.dims_to_reduce=[-2], self.percentile=99.999, self.mode='asymmetric', self.zero_point_bits=16)
      (1): S4BlockRecurrent(
        (layer_activation): QGELU()
        (mult_activation): Identity()
        (drop): Identity()
        (output_linear): Sequential(
          (0): Sequential(
            (0): ActivationQuantizer(self.fake_real='fake', self.num_bits=8, self.dims_to_reduce=None, self.percentile=99.999, self.mode='asymmetric', self.zero_point_bits=16)
            (1): QLinear()
          )
          (1): GLU(dim=-1)
        )
        (state_quantizer): ActivationQuantizer(self.fake_real='fake', self.num_bits=8, self.dims_to_reduce=[-1

## Test with MLP

In [3]:
# Test loading from sweep data

from quantize import set_up_quantizers
from pathlib import Path

fname = r"/home/leo/QuantisedSSM/S4/configs/MLP_config.yaml"
resume_checkpoint = Path(r"/home/leo/QuantisedSSM/S4/log/test") / "checkpoint" / "ckpt.pth"

config_yaml = read_yaml(fname)

# q_config_fname = r"/home/leo/QuantisedSSM/S4/configs/q_config_MLP.yaml"
# q_config_yaml = read_yaml(q_config_fname)
# q_config = ConfigParser(q_config_yaml, resume_checkpoint, modification=None, save_log=False)
# modification = {"run_name": q_config["run_name"]}

modification ={"seed": 0}

config = ConfigParser(config_yaml, None, modification=modification, save_log=False)

q_trainer = set_up_trainer(config)
# set_up_quantizers(q_trainer, q_config)
# q_trainer.load_checkpoint(resume_checkpoint)

q_trainer.save_dir = Path(r"/home/leo/QuantisedSSM/S4/log/test")

q_trainer.model

[*] Generating MNIST Classification Dataset...
[*] Train set size: 50000
[*] Validation set size: 10000
[*] Test set size: 10000
Layer (type:depth-idx)                   Output Shape              Param #
├─Flatten: 1-1                           [-1, 784]                 --
├─MLP: 1-2                               [-1, 10]                  --
|    └─Linear: 2-1                       [-1, 64]                  50,240
|    └─BatchNorm1d: 2-2                  [-1, 64]                  128
|    └─ReLU: 2-3                         [-1, 64]                  --
|    └─Dropout: 2-4                      [-1, 64]                  --
|    └─Linear: 2-5                       [-1, 10]                  650
|    └─Dropout: 2-6                      [-1, 10]                  --
Total params: 51,018
Trainable params: 51,018
Non-trainable params: 0
Total mult-adds (M): 0.10
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.19
Estimated Total Size (MB): 0.20


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): MLP(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): Dropout(p=0.0, inplace=False)
  )
)

In [140]:
# q_trainer.disable_all_quantization()
# q_trainer.train_epoch(1)

# q_trainer.set_static_quantization()

# q_trainer.quantize_all_weights()
# q_trainer.eval()

0it [00:00, ?it/s]

TRAIN - Batch Idx: (97/98) | Loss: 0.349 | Acc: 90.362% (45181/50000): : 98it [00:17,  5.65it/s]
VAL - Batch Idx: (19/20) | Loss: 0.193 | Acc: 94.360% (9436/10000): : 20it [00:03,  5.56it/s]


Best model found. Saving checkpoint at /home/leo/QuantisedSSM/S4/log/test


TEST - Batch Idx: (19/20) | Loss: 0.187 | Acc: 94.430% (9443/10000): : 20it [00:03,  6.10it/s]


{'val_loss': 0.19265839457511902,
 'val_acc': 94.36,
 'test_loss': 0.18653013035655022,
 'test_acc': 94.43,
 'epoch': 1,
 'train_loss': 0.3486754568863888,
 'train_acc': 90.362,
 'lr': [0.009938441702975689]}

In [141]:
q_trainer.calibrate(num_batches=1)
q_trainer.eval()

Starting calibration with 1 batches
Disabling quantization for 0
Disabled quantization for activation quantizer
Disabling quantization for 1
Disabling quantization for 0
Disabled quantization for activation quantizer
Disabling quantization for 1
Set up collecting statistics for activation quantizer
Starting statistics collection for 0
Set up collecting statistics for activation quantizer
Starting statistics collection for 0
Ending statistics collection for 0
Finished collecting statistics for activation quantizer
Updated amax and amin for activation quantizer: max_min: (tensor([[2.8215]], device='cuda:0'), tensor([[-0.4242]], device='cuda:0'))
Reset statistics for activation quantizer
Ending statistics collection for 0
Finished collecting statistics for activation quantizer
Updated amax and amin for activation quantizer: max_min: (tensor([[3.8583]], device='cuda:0'), tensor([[0.]], device='cuda:0'))
Reset statistics for activation quantizer
Enabled quantization for activation quantizer

TEST - Batch Idx: (19/20) | Loss: 0.187 | Acc: 94.430% (9443/10000): : 20it [00:02,  7.25it/s]

Eval Acc: 94.43, Loss: 0.18653013035655022


{'loss': 0.18653013035655022, 'acc': 94.43}

In [145]:
q_trainer.set_static_quantization()
q_trainer.eval()

ValueError: Cannot set static quantization without setting amax/amin. Run calibration first